In [1]:
import sys
sys.path.append('../imprint/research/berry/')
import berrylib.util as util
util.setup_nb()

In [2]:
import jax
import jax.numpy as jnp
import numpy as np
import time
import numpyro
import numpyro.distributions as dist
import scipy.stats
import matplotlib.pyplot as plt
from scipy.special import logit, expit

import inla

/Users/tbent/.mambaforge/envs/imprint/lib/python3.10/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "
/Users/tbent/.mambaforge/envs/imprint/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
narms = 4
mu_0 = -1.34
mu_sig2 = 100.0
sig2_alpha = 0.0005
sig2_beta = 0.000005
logit_p1 = -1.0
def model(data):
    sig2 = numpyro.sample("sig2", dist.InverseGamma(sig2_alpha, sig2_beta))
    cov = jnp.full((narms, narms), mu_sig2) + jnp.diag(jnp.repeat(sig2, narms))
    theta = numpyro.sample(
        "theta",
        dist.MultivariateNormal(mu_0, cov),
    )
    numpyro.sample(
        "y",
        dist.BinomialLogits(theta + logit_p1, total_count=data[:,1]),
        obs=data[:, 0],
    )

In [96]:
data = np.array([[7,35], [6.0,35], [5,35], [4,35]])
params = dict(
    sig2 = 10.0,
    theta = np.array([0.0,0.0,0,0]),
)
ll_fnc = inla.build_raw_log_likelihood(model)
ll_fnc(params, dict(sig2=None, theta=None), data)

DeviceArray(-32.336067, dtype=float32)

In [97]:
sig2_rule = util.log_gauss_rule(15, 1e-2, 1e2)

In [109]:
def conditional(theta, sig2, data):
    params = dict(theta=theta, sig2=sig2)
    return ll_fnc(params, dict(sig2=None, theta=None), data)


def grad_hess(theta, sig2, data):
    p = dict(sig2 = None, theta=theta)
    fixed = dict(sig2=sig2, theta=None)
    p, fixed = fixed, p
    grad = jax.grad(ll_fnc)(p, fixed, data)
    hess = jax.hessian(ll_fnc)(p, fixed, data)
    return grad, hess


grad_hess_vmap = jax.jit(
    jax.vmap(jax.vmap(grad_hess, in_axes=(0, 0, None)), in_axes=(0, None, 0))
)
conditional_vmap = jax.jit(
    jax.vmap(jax.vmap(conditional, in_axes=(0, 0, None)), in_axes=(0, None, 0))
)

In [112]:
for N in 2 ** np.arange(4, 20, 2):
    y = scipy.stats.binom.rvs(35, 0.3, size=(N, narms))
    n = np.full_like(y, 35)
    data = np.stack((y, n), axis=-1)
    theta = np.random.rand(N, sig2_rule.pts.shape[0], narms)
    for i in range(3):
        start = time.time()
        grad, hess = grad_hess_vmap(theta, sig2_rule.pts, data)
        # grad.block_until_ready().shape
        # hess.block_until_ready().shape
        hess['sig2']['sig2'].block_until_ready()
        # hess['theta']['theta'].block_until_ready()
        end = time.time()
    print(f'{N} iters, {(end - start) / N * 1e6:.3f} us per sample, {end - start:.2f}s total')

16 iters, 6.557 us per sample, 0.00s total
64 iters, 5.014 us per sample, 0.00s total
256 iters, 5.859 us per sample, 0.00s total
1024 iters, 4.766 us per sample, 0.00s total
4096 iters, 5.388 us per sample, 0.02s total
16384 iters, 3.837 us per sample, 0.06s total
65536 iters, 3.665 us per sample, 0.24s total
262144 iters, 3.922 us per sample, 1.03s total


In [51]:
def grad_hess(theta, sig2, data):
    p = dict(sig2 = sig2, theta=None)
    fixed = dict(sig2=None, theta=theta)
    grad = jax.grad(ll_fnc)(p, fixed, data)
    hess = jax.hessian(ll_fnc)(p, fixed, data)
    return grad, hess


grad_hess_vmap = jax.jit(
    jax.vmap(jax.vmap(grad_hess, in_axes=(0, 0, None)), in_axes=(0, None, 0))
)
conditional_vmap = jax.jit(
    jax.vmap(jax.vmap(conditional, in_axes=(0, 0, None)), in_axes=(0, None, 0))
)


In [56]:
%%time
grad, hess = grad_hess_vmap(theta, sig2_rule.pts, data)

CPU times: user 2.31 s, sys: 490 ms, total: 2.8 s
Wall time: 1.08 s


In [55]:
grad

{'sig2': DeviceArray([[ 1.26959938e+02,  2.39447601e+02,  3.72434540e+02,  2.70206528e+01, -8.48596764e+00,
               -1.29554920e+01, -5.03462315e+00, -2.44077396e+00, -9.70911443e-01, -4.02829409e-01,
               -1.80530325e-01, -9.00197998e-02, -5.13372608e-02, -3.43636535e-02, -2.73817256e-02],
              [ 6.50315979e+02,  1.18522363e+03,  3.70973419e+02, -2.58413906e+01, -9.72018051e+00,
               -1.04604998e+01, -6.09410572e+00, -2.41068912e+00, -9.77381289e-01, -4.05827969e-01,
               -1.80495828e-01, -9.00461152e-02, -5.13696857e-02, -3.43905315e-02, -2.73903813e-02],
              [ 1.50967822e+03, -8.43325577e+01,  5.37578812e+01, -1.21075859e+01, -1.57736187e+01,
               -1.30084629e+01, -5.75625420e+00, -2.31971765e+00, -9.85959828e-01, -4.05505121e-01,
               -1.81316838e-01, -8.99291337e-02, -5.13091497e-02, -3.43851559e-02, -2.74071805e-02],
              [-4.29113541e+01,  4.93653931e+02,  4.50877960e+02, -8.15600395e+00,  6.998

In [46]:
grad['sig2'].shape, grad['theta'].shape

((262144, 15), (262144, 15, 4))

In [ ]:

n_arms = narms
sigma2_n = sig2_rule.pts.shape[0]
arms = np.arange(n_arms)
cov = np.full((sigma2_n, n_arms, n_arms), mu_sig2)
cov[:, arms, arms] += sig2_rule.pts[:, None]
neg_precQ = -np.linalg.inv(cov)

neg_precQ_b = neg_precQ[:, 0, 1]
neg_precQ_a = neg_precQ[:, 0, 0] - neg_precQ_b

logprecQdet = 0.5 * jnp.log(jnp.linalg.det(-neg_precQ))
log_prior = jnp.array(
    scipy.stats.invgamma.logpdf(sig2_rule.pts, sig2_alpha, scale=sig2_beta)
)
const = log_prior + logprecQdet
na = jnp.arange(narms)

def conditional(theta, sig2, data):
    y = data[..., 0]
    n = data[..., 1]
    theta_m0 = theta - mu_0
    theta_adj = theta + logit_p1
    exp_theta_adj = jnp.exp(theta_adj)
    quad = jnp.sum(
        theta_m0
        * ((theta_m0.sum(axis=-1) * neg_precQ_b)[..., None] + theta_m0 * neg_precQ_a),
        axis=-1,
    )
    return (
        0.5 * quad
        + jnp.sum(
            theta_adj * y[:, None] - n[:, None] * jnp.log(exp_theta_adj + 1),
            axis=-1,
        )
        + const
    )


def grad_hess(theta, _, data):
    y = data[..., 0]
    n = data[..., 1]
    theta_m0 = theta - mu_0
    exp_theta_adj = jnp.exp(theta + logit_p1)
    C = 1.0 / (exp_theta_adj + 1)
    nCeta = n[:, None] * C * exp_theta_adj
    grad = (
        jnp.matmul(neg_precQ[None], theta_m0[:, :, :, None])[..., 0]
        + y[:, None]
        - nCeta
    )
    hess = neg_precQ[None] - ((nCeta * C)[:, :, None, :] * jnp.eye(narms))
    return grad, hess


grad_hess_vmap = jax.jit(grad_hess)
conditional_vmap = jax.jit(conditional)


In [53]:
N = 10000
y = scipy.stats.binom.rvs(35, 0.3, size=(N, narms))
n = np.full_like(y, 35)
data = np.stack((y, n), axis=-1)

In [130]:
b = neg_precQ[0,0,1]
a = neg_precQ[0,0,0] - b

In [131]:
@jax.jit
def quad(theta_max, a, b):
    dotprod = ((theta_max.sum(axis=-1) * b)[..., None] + theta_max * a)
    quad = jnp.sum(theta_max * dotprod, axis=-1)
    return quad

In [132]:
%%timeit
quad(theta_max, a, b).block_until_ready()

126 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [129]:
quad(theta_max, a, b)[0,0]

DeviceArray(-0.37294438, dtype=float32)

In [122]:
dotprod2 = jnp.einsum("...ij,...j", neg_precQ, theta_max)
quad2 = jnp.einsum("...i,...ij,...j", theta_max, neg_precQ, theta_max)
quad3 = np.sum(theta_max * dotprod2, axis=-1)

In [123]:
dotprod[0,0], dotprod2[0,0], quad[0,0], quad2[0,0], quad3[0,0]

(DeviceArray([-5.1011925,  1.39013  ,  2.319191 ,  1.390357 ], dtype=float32),
 DeviceArray([-5.101193 ,  1.3901292,  2.319191 ,  1.390357 ], dtype=float32),
 DeviceArray(-0.3729442, dtype=float32),
 DeviceArray(-0.37294447, dtype=float32),
 DeviceArray(-0.37294444, dtype=float32))

In [55]:
inla_obj = inla.INLA(conditional_vmap, grad_hess_vmap, sig2_rule, narms)

In [56]:
theta_max, hess, iters = inla_obj.optimize_loop(data, sig2_rule.pts, 1e-3)
post = inla_obj.posterior(theta_max, hess, sig2_rule.pts, sig2_rule.wts, data)

In [58]:
%%timeit -n 20 -r 5
theta_max, hess, iters = inla_obj.optimize_loop(data, sig2_rule.pts, 1e-3)
post = inla_obj.posterior(theta_max, hess, sig2_rule.pts, sig2_rule.wts, data)

31.4 ms ± 444 µs per loop (mean ± std. dev. of 5 runs, 20 loops each)
